# Entropy pruning

## IMPORTS

In [2]:
import os
import tempfile

import numpy as np
import pandas as pd

from configparser import ConfigParser
os.environ["TF_USE_LEGACY_KERAS"]="1"

from tf_keras import losses,optimizers, models
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from utils import load_dataset, reformat_image, entropy_prune_model,time_benchmark, normalize_img
from utils.Objects import EntropyPruningSurgeon, PrunedModel
from utils.Plots import plot_entropy_accuracy_threshold

---

## CONSTANTS

In [3]:
config = ConfigParser()
config.read("config.ini")

BATCH_SIZE = int(config["BATCHES"]["BATCH_SIZE"])

IMG_SIZE = (int(config["IMAGES"]["IMG_SIZE"]), int(config["IMAGES"]["IMG_SIZE"]))

MODEL_DIRECTORY = config["MODEL"]["MODEL_DIRECTORY"]
METRICS_DIRECTORY = config["MODEL"]["METRICS_DIRECTORY"]
WEIGHTS_DIRECTORY = config["MODEL"]["WEIGHTS_DIRECTORY"]

ENTROPY_METRICS = os.path.join(METRICS_DIRECTORY, config["MODEL"]["ENTROPY_METRICS"])
HORSE_AND_HUMAN_MODEL = os.path.join(MODEL_DIRECTORY, config["MODEL"]["HORSE_AND_HUMAN_MODEL"])
ENTROPY_HORSE_AND_HUMAN_WEIGHTS = os.path.join(WEIGHTS_DIRECTORY, config["MODEL"]["ENTROPY_HORSE_AND_HUMAN_WEIGHTS"])

## Import the dataset

In [5]:
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Constructing tf.data.Dataset horses_or_humans for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0


In [6]:
train_batches = train_examples.cache().shuffle(num_examples // 4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
# Choose the best entropy level
k_entropies = np.arange(0.019, 0.031, 0.001)
metric_list = []

for k in k_entropies:
    # Load in the best saved model
    model_1 = models.load_model("model_experiments/custom_vgg16.keras")
    pruned_model: PrunedModel = entropy_prune_model(model_1, 20, k, train_batches, validation_batches, BATCH_SIZE)
    metrics = pruned_model.metrics
    val_loss = metrics["val_loss"]
    val_accuracy = metrics["val_accuracy"]

    print(f"Entropy={k}, \t {val_loss=}, \t {val_accuracy=}")
    metric_list.append(metrics)

# Create a dataframe of the values obtained
df = pd.DataFrame(metric_list)
df.to_csv(ENTROPY_METRICS)
df

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 2/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 0/512 channels from layer: block4_conv1
Deleting 0/512 channels from layer: block4_conv2
Deleting 0/512 channels from layer: block4_conv3
Deleting 0/512 channels from layer: block5_conv1
Deleting 0/512 channels from layer: block5_conv2
Deleting 0/512 channels from layer: block5_conv3
Epoch 1/20



 8/23 [=========>....................] - ETA: 46s - loss: 0.6907 - accuracy: 0.9609

In [22]:
data = pd.read_csv(ENTROPY_METRICS)
plot_entropy_accuracy_threshold(data)

FileNotFoundError: [Errno 2] No such file or directory: 'metrics_experiments\\entropy_metrics.csv'

## Final Model Pruning Entropy

In [33]:
test_threshold = 0.028
final_model = models.load_model("model_experiments/custom_vgg16.keras")
entropy = EntropyPruningSurgeon(model=final_model, threshold=test_threshold)
pruned_model = entropy.run()
pruned_model.summary()

Deleting 0/64 channels from layer: block1_conv1
Deleting 0/64 channels from layer: block1_conv2
Deleting 3/128 channels from layer: block2_conv1
Deleting 0/128 channels from layer: block2_conv2
Deleting 0/256 channels from layer: block3_conv1
Deleting 0/256 channels from layer: block3_conv2
Deleting 0/256 channels from layer: block3_conv3
Deleting 374/512 channels from layer: block4_conv1
Deleting 265/512 channels from layer: block4_conv2
Deleting 278/512 channels from layer: block4_conv3
Deleting 280/512 channels from layer: block5_conv1
Deleting 252/512 channels from layer: block5_conv2
Deleting 255/512 channels from layer: block5_conv3
Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 input_40 (InputLayer)       multiple                  0         
     

In [32]:
final_threshold = 0.026
final_model = models.load_model("model_experiments/custom_vgg16.keras")
pruned_model: PrunedModel = entropy_prune_model(final_model, 20, final_threshold, train_batches, validation_batches, BATCH_SIZE)
final_pruned_model = pruned_model.model
final_pruned_model.summary()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000210450C2930>>
Traceback (most recent call last):
  File "C:\Users\neyen\Documents\Pro\Projet\tf1\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\neyen\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1535, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [28]:
# Test the model
final_pruned_model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer = optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
final_pruned_model.evaluate(validation_batches)
final_pruned_model.save_weights(ENTROPY_HORSE_AND_HUMAN_WEIGHTS)

10/10 [==============================] - 68s 6s/step - loss: 0.0026 - accuracy: 1.0000


In [29]:
val_data = validation_examples
val_data = val_data.map(reformat_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_data = val_data.map(normalize_img)

In [38]:
original_model = models.load_model("model_experiments/custom_vgg16.keras")
init_time, avg_time, std = time_benchmark(final_pruned_model, class_names, val_data)

1/1 [==============================] - 3s 3s/step
The first image takes 10724.78 ms
The average time taken per 99 images 5119.70 ms
The standard deviation of samples is 891.79 ms


In [31]:
init_time, avg_time, std = time_benchmark(final_pruned_model, class_names, val_data)

1/1 [==============================] - 5s 5s/step
The first image takes 11302.82 ms
The average time taken per 99 images 7002.33 ms
The standard deviation of samples is 673.59 ms


In [34]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_pruned_model)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmp1oipxvo9\assets


INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmp1oipxvo9\assets


Saved pruned TFLite model to: C:\Users\neyen\AppData\Local\Temp\tmpb9_r9nd8.tflite


In [35]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmpc4k_ldk9\assets


INFO:tensorflow:Assets written to: C:\Users\neyen\AppData\Local\Temp\tmpc4k_ldk9\assets


Saved quantized and pruned TFLite model to: C:\Users\neyen\AppData\Local\Temp\tmpl8qz4mtb.tflite


In [36]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [37]:
print("Size of gzipped baseline quantized pruned model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

# Calculate the difference in size
size_difference = get_gzipped_model_size(quantized_and_pruned_tflite_file) / get_gzipped_model_size(pruned_tflite_file)
print(f"The quantized and pruned model is {size_difference:.2f} times smaller than the pruned model.")

Size of gzipped baseline quantized pruned model: 8535134.00 bytes
Size of gzipped pruned Keras model: 38482075.00 bytes
The quantized and pruned model is 0.22 times smaller than the pruned model.
